In [ ]:
# Check libraries
!pip install transformers

#Required for ROBERTA
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import json
import transformers
import torch
from torch.utils.data import DataLoader
import time

In [ ]:
path = "/content/drive/MyDrive/QA_System/Datasets/XQUAD/final_xquad.json"

with open(path, "rb") as f:
  data = json.load(f)

context = []
questions = []
answers = []
answer_start = []

for a in data:
    for i in a:
        for j in i["paragraphs"]:
            for k in j["qas"]:
                questions.append(k["question"])
                context.append(j["context"])

                for l in k["answers"]:
                    answers.append(l["text"])
                    answer_start.append(l["answer_start"])

train_context, train_question, train_answer, train_start = context, questions, answers, answer_start

In [ ]:
path = "/content/drive/MyDrive/QA_System/Datasets/XQUAD/xquad.hi.json"

with open(path, "rb") as f:
  data = json.load(f)

data = data["data"]

context = []
questions = []
answers = []
answer_start = []

for i in data:
  for j in i["paragraphs"]:
    for k in j["qas"]:
      questions.append(k["question"])
      context.append(j["context"])

      for l in k["answers"]:
        answers.append(l["text"])
        answer_start.append(l["answer_start"])

val_context, val_question, val_answer, val_start = context, questions, answers, answer_start

In [ ]:
answer_end = []
answer_start1 = []
for answer,start, text in zip(train_answer, train_start, train_context):
    real_answer = answer
    start_idx = start

    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters 
    if text[start_idx:end_idx] == real_answer:
      answer_start1.append(start_idx)
      answer_end.append(end_idx)

    # When the real answer is more by one character -> white spaces, special char
    elif text[start_idx-1:end_idx-1] == real_answer:
      answer_start1.append(start_idx-1)
      answer_end.append(end_idx-1) 

    # When the real answer is more by two characters  
    elif text[start_idx-2:end_idx-2] == real_answer:
      answer_start1.append(start_idx-2)
      answer_end.append(end_idx-2)

train_end, train_start = answer_end, answer_start1


In [ ]:
answer_end = []
answer_start1 = []
for answer,start, text in zip(val_answer, val_start, val_context):
    real_answer = answer
    start_idx = start

    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters 
    if text[start_idx:end_idx] == real_answer:
      answer_start1.append(start_idx)
      answer_end.append(end_idx)

    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
      answer_start1.append(start_idx-1)
      answer_end.append(end_idx-1) 

    # When the real answer is more by two characters  
    elif text[start_idx-2:end_idx-2] == real_answer:
      answer_start1.append(start_idx-2)
      answer_end.append(end_idx-2)


val_end, val_start = answer_end, answer_start1



In [ ]:
# Import tokenizer and optimizers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AdamW

In [ ]:
model_name = "xlm-roberta-large"

In [ ]:
# Downloading ROBERTA configuration
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream tas

In [ ]:
# Downloading tokenizer -> splitting sentence into words
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [ ]:
# Tokenizing contexts and questions
train_encodings = tokenizer(train_context, train_question, truncation=True, padding=True)
val_encodings = tokenizer(val_context, val_question, truncation=True, padding=True)

In [ ]:
# Computing start and end index of answers in tokenized data
def add_token_positions(encodings, answers, start, end):
  start_positions = []
  end_positions = []

  count = 0

  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, start[i]))
    end_positions.append(encodings.char_to_token(i, end[i]))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      #start_positions[-1] = tokenizer.model_max_length
        start_positions[-1] = 512
      
    # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
    if end_positions[-1] is None:
      end_positions[-1] = encodings.char_to_token(i, end[i] - 1)
      # if end position is still None the answer passage has been truncated
      if end_positions[-1] is None:
        count += 1
        #end_positions[-1] = tokenizer.model_max_length
        end_positions[-1] = 512



  # Update the data in dictionary
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answer, train_start, train_end)
add_token_positions(val_encodings, val_answer, val_start, val_end)

In [ ]:
# Creating Dataset Class
class XquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
#Creating dataset objects -> Converting nomrnal dataset to pytorch compatible dataset 
train_dataset = XquadDataset(train_encodings)
val_dataset = XquadDataset(val_encodings)

In [ ]:
# Data loader combines a dataset and a sampler and provides an iterable over the given dataset.
train_loader = DataLoader(dataset = train_dataset, batch_size=2, shuffle=False, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)

In [ ]:
i=0
for x,y in enumerate(train_loader):
  print(x,y)
  if i==0:
    break

0 {'input_ids': tensor([[    0,  3108, 14603,  ...,     1,     1,     1],
        [    0,  3108, 14603,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'start_positions': tensor([ 11, 171]), 'end_positions': tensor([ 12, 171])}


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available()
                        else 'cpu')

In [ ]:
# Transforming configurations to available device
model1 = AutoModelForQuestionAnswering.from_pretrained(model_name1).to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream tas

In [ ]:
optim = AdamW(model.parameters(), lr=3e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 5

In [ ]:
# Actual Fine Tuning of BERT
whole_train_eval_time = time.time()

train_losses = []
val_losses = []

print_every = 100

for epoch in range(epochs):
  epoch_time = time.time()

  # Set model in train mode
  model.train()
    
  loss_of_epoch = 0

  print("############Train############")

  for batch_idx,batch in enumerate(train_loader): 
    
    optim.zero_grad()

    # batch = context[SEP]question[SEP]answer[SEP]start[SEP]end[SEP]
    # input ids are mapping of tokens with id
    
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    # do a backwards pass 
    loss.backward()
    # update the weights -> According to error
    optim.step()
    # Find the total loss 
    loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
      print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(train_loader)
  train_losses.append(loss_of_epoch)

  ##########Evaluation##################

  # Set model in evaluation mode
  model.eval()

  print("############Evaluate############")

  loss_of_epoch = 0

  for batch_idx,batch in enumerate(val_loader):
    
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss = outputs[0]
      # Find the total loss
      loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
       print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(val_loader)
  val_losses.append(loss_of_epoch)

  # Print each epoch's time and train/val loss 
  print("\n-------Epoch ", epoch+1,
        "-------"
        "\nTraining Loss:", train_losses[-1],
        "\nValidation Loss:", val_losses[-1],
        "\nTime: ",(time.time() - epoch_time),
        "\n-----------------------",
        "\n\n")


print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(15,10))

ax.set_title("Train and Validation Losses",size=20)
ax.set_ylabel('Loss', fontsize = 20) 
ax.set_xlabel('Epochs', fontsize = 25) 
_=ax.plot(train_losses) 
_=ax.plot(val_losses) 
_=ax.legend(('Train','Val'),loc='upper right')

In [ ]:
# Define the bert tokenizer and load fined tuned model
from transformers import AutoTokenizer,BertTokenizerFast

tokenizer = AutoTokenizer.from_pretrained(model_name1)
path = f"/content/drive/MyDrive/XQUAD/xlm_roberta_large5.pt"
model=torch.load(path,map_location=torch.device('cpu'))
model.eval()

In [ ]:
# Function to predict answer based on provided context and query
def predict(context,query):

  # Encode query and context 
  inputs = tokenizer.encode_plus(query, context, padding=True, max_length= 512, truncation=True, add_special_tokens = True, return_tensors='pt')

  # Generate output
  outputs = model(**inputs)

  # Extract start and end index 
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1 

  # Decode answer from tokenized form
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

# Normalize text -> Remove articles, spaces, punctuation marks and conversion to all upper cases to lower cases
def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

# Compute Exact Match Score (em score)
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

#Compute F1 score
def compute_f1(prediction, truth):
  # Normalizing predicted and true answer
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  # Intersection of Predicted and true answer
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  # Ratio of length of common tokens to complete tokens length
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  # Return Harmonic Mean
  return 2 * (prec * rec) / (prec + rec)

In [ ]:
# Predicting answer and returning metrics of evaluation 
def give_an_answer(context,query,answer):

  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  
  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print("\n")

  return prediction

In [ ]:
context = '''Hi! Mein Name ist Alexa und ich bin 21 Jahre alt. Ich habe gelebt
            in Peristeri von Athen , aber jetzt zog ich weiter in Kaisariani von Athen.'''

queries = ["Wie alt ist Alexa?",
           "Wo hat Alexa früher gelebt?"
          ]
answers = ["21",
           "Peristeri of Athens"
          ]
s=[]
for q,a in zip(queries,answers):
  #give_an_answer(context,q,a)
  (give_an_answer(context,q,a))



Question: Wie alt ist Alexa?
Prediction: 21
True Answer: 21
EM: 1
F1: 1.0


Question: Wo hat Alexa früher gelebt?
Prediction: Peristeri von Athen
True Answer: Peristeri of Athens
EM: 0
F1: 0.3333333333333333




In [ ]:
f1 = 0
em = 0
for c,q,a, in zip(val_context, val_question, val_answer):
  res = give_an_answer(c,q,a)
  f1 += res["f1"]
  em += res["em"]

In [ ]:
f1_score = (f1*100)//(len(val_context))
em_score = (em*100)//(len(val_context))

print("F1 score : ", f1_score, "\nEM score : ", em_score)

In [ ]:
f1 = 0
em = 0
for c,q,a, in zip(train_context, train_question, train_answer):
  res = give_an_answer(c,q,a)
  f1 += res["f1"]
  em += res["em"]

In [ ]:
f1_score = (f1*100)//(len(train_context))
em_score = (em*100)//(len(train_context))

print("F1 score : ", f1_score, "\nEM score : ", em_score)